In [1]:
import pandas as pd
import random as rd
from itertools import combinations
import math

class TS():
    def __init__(self, Path, seed, tabu_tenure):
        self.Path = Path
        self.seed = seed
        self.tabu_tenure = tabu_tenure
        self.instance_dict = self.input_data()
        self.Initial_solution = self.get_InitialSolution()
        self.tabu_str, self.Best_solution, self.Best_objvalue = self.TSearch()
    
    def input_data(self):
        return pd.read_excel(self.Path, names=['Job', 'weight', 'processing_time', 'due_date'], index_col=0).to_dict('index')
    
    def get_tabuestructure(self):
        dict = {}
        for swap in combinations(self.instance_dict.keys(), 2):
            dict[swap] = {'tabu_time': 0, 'MoveValue': 0}
        return dict
    
    def get_InitialSolution(self):
        n_jobs = len(self.instance_dict)
        initial_solution = list(range(1, n_jobs + 1))
        rd.seed(self.seed)
        rd.shuffle(initial_solution)
        return initial_solution
    
    def Objfun(self, solution):
        dict = self.instance_dict
        t = 0
        objfun_value = 0
        for job in solution:
            C_i = t + dict[job]['processing_time']
            d_i = dict[job]['due_date']
            T_i = max(0, C_i - d_i)
            W_i = dict[job]['weight']
            objfun_value += W_i * T_i
            t = C_i
        return objfun_value
    
    def SwapMove(self, solution, i, j):
        solution = solution.copy()
        i_index = solution.index(i)
        j_index = solution.index(j)
        solution[i_index], solution[j_index] = solution[j_index], solution[i_index]
        return solution
    
    def TSearch(self):
        tenure = self.tabu_tenure
        tabu_structure = self.get_tabuestructure()
        best_solution = self.Initial_solution
        best_objvalue = self.Objfun(best_solution)
        current_solution = self.Initial_solution
        current_objvalue = self.Objfun(current_solution)
        iter = 1
        Terminate = 0
        
        while Terminate < 100:
            if iter <= 10:
                print(f'Iteration {iter}: Best_objvalue: {best_objvalue}')
            
            for move in tabu_structure:
                candidate_solution = self.SwapMove(current_solution, move[0], move[1])
                candidate_objvalue = self.Objfun(candidate_solution)
                tabu_structure[move]['MoveValue'] = candidate_objvalue
            
            while True:
                best_move = min(tabu_structure, key=lambda x: tabu_structure[x]['MoveValue'])
                MoveValue = tabu_structure[best_move]['MoveValue']
                tabu_time = tabu_structure[best_move]['tabu_time']
                
                if tabu_time < iter:
                    current_solution = self.SwapMove(current_solution, best_move[0], best_move[1])
                    current_objvalue = self.Objfun(current_solution)
                    
                    if MoveValue < best_objvalue:
                        best_solution = current_solution
                        best_objvalue = current_objvalue
                        Terminate = 0
                    else:
                        Terminate += 1
                    
                    tabu_structure[best_move]['tabu_time'] = iter + tenure
                    iter += 1
                    break
                else:
                    if MoveValue < best_objvalue:
                        current_solution = self.SwapMove(current_solution, best_move[0], best_move[1])
                        current_objvalue = self.Objfun(current_solution)
                        best_solution = current_solution
                        best_objvalue = current_objvalue
                        Terminate = 0
                        iter += 1
                        break
                    else:
                        tabu_structure[best_move]['MoveValue'] = float('inf')
                        continue
        
        print(f'\nTabu search completed')
        print(f'\nPerformed iterations: {iter}')
        print(f'Best found Solution: {best_solution}')
        print(f'Best found Objvalue: {best_objvalue}')
        return tabu_structure, best_solution, best_objvalue

print(f"Starting Tabu search\n")
test = TS(Path="Instance_10.xlsx", seed=2012, tabu_tenure=3)

Starting Tabu search



FileNotFoundError: [Errno 2] No such file or directory: 'Instance_10.xlsx'

In [1]:

import numpy as np

class TabuSearch:
    def __init__(self,fitness_function,initial_solution,tabu_list_size=10,max_iter=100):
        self.fitness_function = fitness_function
        self.initial_solution = initial_solution
        self.tabu_list = []
        self.tabu_list_size = tabu_list_size
        self.max_iter = max_iter
        self.current_solution = initial_solution
        self.best_solution = initial_solution
        self.best_solution_value = self.fitness_function(self.best_solution)

    def generate_neighbour(self,solution):
        return [solution+4,solution-4]

    def apply_tabu_restriction(self,neighbours,tabu_list):
        return [neighbour for neighbour in neighbours if neighbour not in tabu_list]

    def select_neighbour(self,neighbours):
        best_neighbour = None
        best_neighbour_value = float('inf')

        for neighbour in neighbours:
            neighbour_value = self.fitness_function(neighbour)
            if neighbour_value < best_neighbour_value:
                best_neighbour = neighbour
                best_neighbour_value = neighbour_value

        return best_neighbour,best_neighbour_value

    def update_tabu_list(self,tabu_list,move,tabu_list_size):
        tabu_list.append(move)
        if len(tabu_list) > tabu_list_size:
            tabu_list.pop(0)

    def run(self):
        for iter in range(self.max_iter):
            neighbours = self.generate_neighbour(self.current_solution)

            allowed_neighbours = self.apply_tabu_restriction(neighbours,self.tabu_list)

            best_neighbour,best_neighbour_value = self.select_neighbour(allowed_neighbours)

            if best_neighbour_value < self.best_solution_value:
                self.best_solution = best_neighbour
                self.best_solution_value = best_neighbour_value
                print(f"Iteration : {iter+1} New best solution found: x = {self.best_solution}, f(x) = {self.best_solution_value}")
            
            self.update_tabu_list(self.tabu_list,best_neighbour,self.tabu_list_size)

            self.current_solution = best_neighbour

            if best_neighbour_value == 0:
                break

        print(f"List : {self.tabu_list} New best solution found: x = {self.best_solution}, f(x) = {self.best_solution_value}")
        # return 

def objective_function(x):
    return x**2

initial_solution = np.random.randint(-100,100)
ts = TabuSearch(objective_function,initial_solution)
ts.run()

Iteration : 1 New best solution found: x = 20, f(x) = 400
Iteration : 2 New best solution found: x = 16, f(x) = 256
Iteration : 3 New best solution found: x = 12, f(x) = 144
Iteration : 4 New best solution found: x = 8, f(x) = 64
Iteration : 5 New best solution found: x = 4, f(x) = 16
Iteration : 6 New best solution found: x = 0, f(x) = 0
List : [20, 16, 12, 8, 4, 0] New best solution found: x = 0, f(x) = 0
